### 스타벅스 매장 입지 분석
- 순서
    1. 데이터 수집
    2. 데이터 전처리
    3. 분석
    4. 시각화

#### 데이터 수집
- 셀레니움 자동화 + 뷰티플수프 정제

In [1]:
# 필요 라이브러리 사용
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [24]:
# 분석, 시각화 라이브러리 사용
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams, rc, font_manager
import warnings
import seaborn as sns
import time

In [25]:
# matplotlib 한글 및 기타 설정

## 맑은 고딕 사용
font_path = 'C:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name() # 폰트 이름 : Malgun Gothic
warnings.simplefilter('ignore')

## 맷플롯립 설정
plt.rcParams['font.family'] = font_name        # 폰트 설정
plt.rcParams['font.size'] = 12                 # 글자 크기
plt.rcParams['figure.figsize'] = (12, 5)       # (W, H)
plt.rcParams['axes.grid'] = True               # 차트 가로세로줄 표시
plt.rcParams['axes.unicode_minus'] = False     # 한글설정 후 '마이너스 깨짐' 방지 

## 시본 설정
sns.set_theme(font=font_name, style='darkgrid', rc={'axes.unicode_minus':False})

### 셀레니움 사용

In [31]:
# 웹드라이버
driver = webdriver.Chrome()

In [32]:
# 스타벅스 코리아사이트 연결
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=quick'
driver.get(url)

In [39]:
# 매장찾기 클릭
## header.find_store_header > h2.btn_find_store > a
link_path = 'header.loca_search > h3 > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()

In [40]:
# 매장찾기 웹소스에서 '지역검색' 링크 검색 후 링크 클릭
## F12 개발자도구로 HTML 태그 정보 확인
## header.loca_search > h3 > a 에 지역검색 링크 존재
link_path = 'header.loca_search > h3 > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()

In [41]:
# 지역검색에서 서울 링크 클릭
## ul.sido_arae_box > li:nth-child(1) > a
link_path = 'ul.sido_arae_box > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()

In [42]:
# 서울에서 전체 링크 클릭
## ul.gugun_arae_box > li:nth-child(1) > a
link_path = 'ul.gugun_arae_box > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()
# 매장 전체 조회가 최초 2~3초 정도 시간이 걸리기 때문에 딜레이를 걸어줌
time.sleep(3.0)

- 메가커피, 빽다방, 컴포즈커피 등 크롤링 가능한 사이트도 동일하게 진행

### 서울 스타벅스 매장 정보 가져오기

In [47]:
html2 = driver.page_source

In [48]:
soup = BeautifulSoup(html2, 'html.parser')

In [49]:
# 서울, 전체 결과에서 
## ul.quickSearchResultBoxSidoGugun > li.quickResultLstCon 만 가져오면 됨
quickResultLst = soup.select('li.quickResultLstCon')
len(quickResultLst)

637

In [51]:
# 각 결과 확인, 부산 현재위치에서 나온 23개 결과가 서울과 중첩
quickResultLst[23]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

##### 판다스 데이터 처리

In [54]:
quickResultLst[630].select('strong')[0].text

'중랑역  '

In [ ]:
# 서울매장 리스트 DF
seoulStores = []

for item in quickResultLst:
    